<a href="https://colab.research.google.com/github/monanjo123/imdb_sentiment_analysis/blob/master/IMDB_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, GlobalAveragePooling1D, LSTM

Using TensorFlow backend.


In [2]:
# Load IMDB dataset

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=10000,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
print(x_train[0], y_train[0])

17465344/17464789 [==============================] - 2s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178,

In [3]:
# Convert Integers back to words

# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
  
decode_review(x_train[1])

1646592/1641221 [==============================] - 1s 1us/step


"<START> big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal <UNK> the hair is big lots of boobs <UNK> men wear those cut <UNK> shirts that show off their <UNK> sickening that men actually wore them and the music is just <UNK> trash that plays over and over again in almost every scene there is trashy music boobs and <UNK> taking away bodies and the gym still doesn't close for <UNK> all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [4]:
# Sequence Padding

x_train = pad_sequences(x_train, maxlen=100, value=0.)
x_test = pad_sequences(x_test, maxlen=100, value=0.)

print(len(x_train), len(x_test))

25000 25000


In [5]:
# build model
vocab_size = 10000
batch_size = 16
epochs = 40

model = Sequential()
model.add(Embedding(vocab_size, batch_size))
model.add(LSTM(500, return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Activation('relu'))


model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
print(model.summary())

W0901 15:25:47.182031 139675243480960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0901 15:25:47.219594 139675243480960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0901 15:25:47.225748 139675243480960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0901 15:25:47.842751 139675243480960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0901 15:25:47.850210 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 500)         1034000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 500)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                8016      
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)               

In [7]:
# Train Model
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(x_train, 
                    y_train,
                    batch_size=512,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True
                   )
# model.save('/drive/My Drive/Machine Learning/Models/cat_dog_classifier_data_augmented.h5')

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 25000 samples, validate on 25000 samples
Epoch 1/40
25000/25000 [==============================] - 14s 575us/step - loss: 0.6205 - acc: 0.7376 - val_loss: 0.5760 - val_acc: 0.7245
Epoch 2/40
25000/25000 [==============================] - 14s 575us/step - loss: 0.5618 - acc: 0.7876 - val_loss: 0.4940 - val_acc: 0.7519
Epoch 3/40
25000/25000 [==============================] - 14s 579us/step - loss: 0.5131 - acc: 0.8183 - val_loss: 0.5320 - val_acc: 0.8076
Epoch 4/40
25000/25000 [==============================] - 15s 582us/step - loss: 0.4429 - acc: 0.8406 - val_loss: 0.4080 - val_acc: 0.8246
Epoch 5/40
25000/25000 [==============================] - 15s 585us/step - loss: 0.4118 - acc: 0.8558 - val_loss: 0.4019 - val_acc: 0.8258
Epoch 6/40
25000/25000 [==============================] - 15s 589us/step - loss: 0.3596 - acc: 0.8725 - val_loss: 0.4101 - val_acc: 0.8104
Epoch 7/40
25000/25000 [==============================] - 15s 592us/step - loss: 0.3244 - acc: 0.8850 - val_loss: 0.